In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
test = pd.read_csv("test.csv")
stores = pd.read_csv("stores.csv")
oil = pd.read_csv("oil.csv")
holidays = pd.read_csv("holidays_events.csv")
train = pd.read_csv("train.csv")

In [3]:
# Convert oil['date'] to datetime
oil['date'] = pd.to_datetime(oil['date'])


In [4]:

# Interpolate missing values
oil['dcoilwtico'] = oil['dcoilwtico'].interpolate(method='linear')

In [5]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [6]:
# Convert date in test set
test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d')

In [7]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           28512 non-null  int64         
 1   date         28512 non-null  datetime64[ns]
 2   store_nbr    28512 non-null  int64         
 3   family       28512 non-null  object        
 4   onpromotion  28512 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1.1+ MB


In [9]:
# Already converted oil['date'] above
# Convert holidays_events['date']
holidays['date'] = pd.to_datetime(holidays['date'])

In [10]:
holidays

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [22]:
train['date'] = pd.to_datetime(train['date'], format='%Y-%m-%d')


# we have merge with main train dataset

In [11]:
# Merge oil and holidays on 'date'
oil_holidays = pd.merge(oil, holidays, on='date', how='outer')

# Perform a cross join between stores and oil_holidays
stores['key'] = 1
oil_holidays['key'] = 1

merged_df = pd.merge(stores, oil_holidays, on='key').drop('key', axis=1)

ValueError: You are trying to merge on object and datetime64[ns] columns for key 'date'. If you wish to proceed you should use pd.concat

In [ ]:
merged_df

In [ ]:
# Add a column to flag if the holiday is relevant for that store
merged_df['is_relevant_holiday'] = (
    ((merged_df['locale'] == 'National')) |
    ((merged_df['locale'] == 'Regional') & (merged_df['state'] == merged_df['locale_name'])) |
    ((merged_df['locale'] == 'Local') & (merged_df['city'] == merged_df['locale_name']))
)

In [ ]:
df = merged_df.copy()

In [ ]:
# Extract standard time features
df['day'] = df['date'].dt.day
df['week'] = df['date'].dt.isocalendar().week.astype(int)
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
# df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Tuesday=1,.... Saturday=5, Sunday=6

# we have done Sunday=1, Monday=2, Tuesday=3 .. till Saturday=7
df['day_of_week'] = ((df['date'].dt.dayofweek + 1) % 7) + 1
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)  # 1 if Sat/Sun, else 0
df['weekday_name'] = df['date'].dt.day_name()

In [ ]:
df

In [ ]:
df['dcoilwtico'].unique()

In [ ]:
df['dcoilwtico'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
holidays_events